In [1]:
import PyPDF2
import re
import pandas as pd

In [2]:
file_paths = [
    './documents/नेपालको-अन्तरिम-संविधान-२०६३.pdf',
    './documents/नेपालको-स_ंविधान.pdf'
]

In [3]:
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

In [4]:
texts = [extract_text_from_pdf(file_path) for file_path in file_paths]


In [13]:
texts

['www.lawcommission.gov.np \nwww.lawcommission.gov.np \n \n1 g]kfnsf] cGtl/d ;+ljwfg, @)^# g]kfnsf] cGtl/d ;+ljwfg, @)^# g]kfnsf] cGtl/d ;+ljwfg, @)^# g]kfnsf] cGtl/d ;+ljwfg, @)^#    \nk|df0fLs/0f / k|sfzg k|df0fLs/0f / k|sfzg k|df0fLs/0f / k|sfzg k|df0fLs/0f / k|sfzg    ldlt ldlt ldlt ldlt    \n                                    @)^#÷!)÷! @)^#÷!)÷! @)^#÷!)÷! @)^#÷!)÷!    \n;+zf]wg ;+zf]wg ;+zf]wg ;+zf]wg        \n!= g]kfnsf] cGtl/d ;+ljwfg -klxnf] ;+zf]wg_,@)^#    @)^#÷! !÷#) \n@= g]kfnsf] cGtl/d ;+ljwfg -bf];|f] ;+zf]wg_, @)^$    @)^$÷ @÷#) \n#= g]kfnsf] cGtl/d ;+ljwfg -t];|f] ;+zf]wg_, @)^$    @)^$÷)( ÷!# \n$= g]kfnsf] cGtl/d ;+ljwfg -rf}yf]] ;+zf]wg_, @)^%    @)^%÷)@ ÷!^ \n%= g]kfnsf] cGtl/d ;+ljwfg -kfrf}] ;+zf]wg_, @)^%    @)^%÷)# ÷@( \n^=  g]kfnsf] cGtl/d ;+ljwfg -5}]7f}} ;+zf]wg_, @)^%    @)^% ÷)*÷#) \n&= g]kfnsf] cGtl/d ;+ljwfg -;ftf}+ ;+zf]wg_, @)^^    @)^^÷! )÷!& \n*= g]kfnsf] cGtl/d ;+ljwfg -cf7f}}+ ;+zf]wg_, @)^&    @)^& ÷)@÷!$ \n(= g]kfnsf] cGtl/d ;+ljwfg -gjf+}} ;+zf]w

In [5]:
def preprocess_and_structure_text(text):
    start_index = text.find("भाग")
    if start_index != -1:
        text = text[start_index:]
    
    # Remove specific URL
    text = text.replace('https://lawcommission.gov.np/np/', '')

    # Remove URLs and unnecessary characters
    text = re.sub(r'\[.*?\]', '', text) 
    text = re.sub(r'http\S+', '', text) 
    text = text.replace('\n', ' ').replace('\r', '')

    # Split text by "भाग"
    bhag_sections = re.split(r'(भाग[–\-]\d+)', text)
    structured_data = []
    current_bhag = ""
    
    for i in range(1, len(bhag_sections), 2):
        current_bhag = bhag_sections[i].strip()
        bhag_text = bhag_sections[i + 1].strip()
        
        lines = bhag_text.split(' ')
        current_header = ""
        current_section_number = ""
        current_text = ""
        
        for line in lines:
            line = line.strip()
            if re.match(r'^[०-९]+\.', line):  # Match headers like १. २. ३.
                if current_text:
                    structured_data.append([current_bhag, current_header, current_section_number, current_text.strip()])
                current_header = line
                current_section_number = ""
                current_text = ""
            elif re.match(r'^\(\d+\)', line):  # Match sub-sections like (१) (२) (३)
                if current_text:
                    structured_data.append([current_bhag, current_header, current_section_number, current_text.strip()])
                current_section_number = line
                current_text = ""
            else:
                current_text += " " + line
        
        if current_text:
            structured_data.append([current_bhag, current_header, current_section_number, current_text.strip()])

    return structured_data

In [6]:
structured_texts = [preprocess_and_structure_text(text) for text in texts]

combined_data = [item for sublist in structured_texts for item in sublist]

filtered_data = [row for row in combined_data if 'http' not in row[3] and 'https://lawcommission.gov.np/np/' not in row[3]]

In [7]:
columns = ["bhag", "header", "section_number", "text"]

In [8]:
df = pd.DataFrame(combined_data, columns=columns)
df.to_csv('./data/judicial_support_system_dataset.csv', index=False)

print("CSV file created successfully.")

CSV file created successfully.


In [9]:
df = pd.read_csv('./data/judicial_support_system_dataset.csv')

In [10]:
df.shape

(1871, 4)

In [11]:
df.head()

,bhag,header,section_number,text
0,भाग–१,NaN,NaN,प्रारजम्भक
1,भाग–१,१.,NaN,संविधान मूल कानूनः
2,भाग–१,१.,(१),यो संविधान नेपालको मूल कानून िो । यस संविधानसँ...
3,भाग–१,१.,(२),यस संविधानको पालना गनुव प्रत्येक व्यजक्तको किव...
4,भाग–१,२.,NaN,सािवभौमसत्ता र राजकीयसत्ताः नेपालको सािवभौमस...
